在gensim 中已经基于fastText 即成来c++ 和 python 接口的版本，我们可以直接使用gensim fasttext <br>

目的： 通过fasttext库 需要学习的内容 <br>
1. training word－embedding models  <br>
2. saving & loading models 
3. performing simililary operations & vector  <br>
4. fasttext vs word2vec  <br>


[1]gensim 官方网站 <br>
https://radimrehurek.com/gensim/models/fasttext.html  <br>
[2]fastText word vector-Enriching word vector with Subword information <br>
https://arxiv.org/abs/1607.04606 <br>
[3]fastText by gensim version . notebook tutorial <br>
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/FastText_Tutorial.ipynb <br>
[4]gensim-word2vec <br>
http://www.52nlp.cn/tag/gensim-word2vec

## 数据预处理

In [73]:
import pandas as pd
import jieba

#停用词
stopwords_data_path = "data/stopwords.txt"
stopwords_data_df = pd.read_csv(stopwords_data_path,encoding="utf-8",sep="\t",index_col=None,quoting=3,names=["stopword"])

# 准备数据
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_sports = df_sports.dropna()

stopwords = stopwords_data_df.stopword.values.tolist()
technology = df_technology.content.apply(lambda line: line.strip()).values.tolist()
car = df_car.content.apply(lambda line: line.strip()).values.tolist()
entertainment = df_entertainment.content.apply(lambda line: line.strip()).values.tolist()
military = df_military.content.apply(lambda line: line.strip()).values.tolist()
sports = df_sports.content.apply(lambda line: line.strip()).values.tolist()

# 分词和中文处理
'''
5类数据最终处理的格式：
word1 word2 word3 technology 
word1 word2 word3 word4 technology 
word1 word2 word4 car 
'''

'''
    lines: 文章的一条记录
    sentences: 返回的数据列表
    category： 文章的类别
'''
def preprocess_text(lines, documents):
    for line in lines:
        segs = jieba.lcut(line)
        segs = [seg for seg in segs if len(seg) > 1 and seg not in stopwords]
        data = " ".join(segs)
        documents.append(data)

# 生成fasttext 无监督学习的样本数据
documents = []
preprocess_text(technology, documents)
preprocess_text(entertainment, documents)
preprocess_text(car, documents)
preprocess_text(military, documents)
preprocess_text(sports, documents)

# ## 样本顺序打乱 并打印训练格式的样本
import random
random.shuffle(documents)

print("documents size = {0}".format(len(documents)))
print("writing data to fasttext unsupervised learning format...")
data_path = "data_sample/fasttext_unsupervised_train_data.txt"
## TODO 存储文件需要utf-8
with open(data_path,"w") as f_write:
    for document in documents:
        line = document.strip()
        f_write.write(line + "\n")
print("done!")

documents size = 125836
writing data to fasttext unsupervised learning format...
done!


In [7]:
import gensim
import logging
import multiprocessing
from time import time
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
sentences=[]
'''
    >>> from gensim.models import Word2Vec
    >>> sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
    >>> model = Word2Vec(sentences, min_count=1)
'''
data_path = "data_sample/fasttext_unsupervised_train_data.txt"
with open(data_path,"r",encoding='UTF-8') as f:
    lines = f.readlines()
    print("load total sample size = {0}".format(len(lines)))
    for line in lines:
        line_lst = line.strip().split(" ")
        sentences.append(line_lst)

load total sample size = 125838


In [8]:
sentences[0:2]

[['中新网',
  '长兴',
  '日电',
  '王逸飞',
  '这是',
  '美妙',
  '体验',
  '中国',
  '参加',
  '比赛',
  '一种',
  '中国',
  '风景',
  '第一个',
  '太湖',
  '图影',
  '国际',
  '半程',
  '马拉松赛',
  '终点',
  '肯尼亚',
  '运动员',
  '拉蒙',
  '享受',
  '比赛'],
 ['疯岳',
  '佳人',
  '岳云鹏',
  '痴情',
  '护工',
  '身份',
  '惊喜',
  '回归',
  '偶遇',
  '少时',
  '女神',
  '袁姗姗',
  '萌生',
  '追爱',
  '愿望',
  '背后',
  '爱情',
  '操作',
  '助攻',
  '更是',
  '信心',
  '大涨',
  '誓要',
  '虏获',
  '女神',
  '芳心',
  '一同',
  '曝光',
  '剧照',
  '两人',
  '甜蜜',
  '虐狗',
  '细节',
  '展露',
  '温馨',
  '进餐',
  '把酒言欢',
  '卖萌',
  '耍宝',
  '搞怪',
  '自拍']]

## word2vec 模型训练

In [10]:
from gensim.models.word2vec import Word2Vec
begin = time()
print("training word2vec model start....")
model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
print("training word2vec model end....")
model.save("model/word2vec_gensim")
model.wv.save_word2vec_format("model/word2vec_org", "model/vocabulary", binary=False)
end = time()
print("Total procesing time: %d seconds" % (end - begin))

2019-03-05 14:34:53,360 : INFO : collecting all words and their counts
2019-03-05 14:34:53,361 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-05 14:34:53,463 : INFO : PROGRESS: at sentence #10000, processed 279869 words, keeping 48298 word types
2019-03-05 14:34:53,536 : INFO : PROGRESS: at sentence #20000, processed 559538 words, keeping 70601 word types


training word2vec model start....


2019-03-05 14:34:53,624 : INFO : PROGRESS: at sentence #30000, processed 838285 words, keeping 86966 word types
2019-03-05 14:34:53,710 : INFO : PROGRESS: at sentence #40000, processed 1116414 words, keeping 100453 word types
2019-03-05 14:34:53,787 : INFO : PROGRESS: at sentence #50000, processed 1394624 words, keeping 111713 word types
2019-03-05 14:34:53,865 : INFO : PROGRESS: at sentence #60000, processed 1667450 words, keeping 121701 word types
2019-03-05 14:34:53,950 : INFO : PROGRESS: at sentence #70000, processed 1944698 words, keeping 130560 word types
2019-03-05 14:34:54,033 : INFO : PROGRESS: at sentence #80000, processed 2223065 words, keeping 138782 word types
2019-03-05 14:34:54,114 : INFO : PROGRESS: at sentence #90000, processed 2499529 words, keeping 146573 word types
2019-03-05 14:34:54,197 : INFO : PROGRESS: at sentence #100000, processed 2775387 words, keeping 154030 word types
2019-03-05 14:34:54,279 : INFO : PROGRESS: at sentence #110000, processed 3050148 words, 

training word2vec model end....


2019-03-05 14:35:16,735 : INFO : saved model/word2vec_gensim
2019-03-05 14:35:16,736 : INFO : storing vocabulary in model/vocabulary
2019-03-05 14:35:16,969 : INFO : storing 52514x100 projection weights into model/word2vec_org


Total procesing time: 27 seconds


## 加载模型并预测

In [11]:
model = Word2Vec.load("model/word2vec_gensim")

2019-03-05 14:35:38,049 : INFO : loading Word2Vec object from model/word2vec_gensim
2019-03-05 14:35:38,505 : INFO : loading wv recursively from model/word2vec_gensim.wv.* with mmap=None
2019-03-05 14:35:38,507 : INFO : setting ignored attribute vectors_norm to None
2019-03-05 14:35:38,508 : INFO : loading vocabulary recursively from model/word2vec_gensim.vocabulary.* with mmap=None
2019-03-05 14:35:38,509 : INFO : loading trainables recursively from model/word2vec_gensim.trainables.* with mmap=None
2019-03-05 14:35:38,511 : INFO : setting ignored attribute cum_table to None
2019-03-05 14:35:38,513 : INFO : loaded model/word2vec_gensim


### 计算相关词

In [12]:
word_similar = model.wv.most_similar("中国",topn=10)
for wv in word_similar:
    print(wv)

2019-03-05 14:35:39,601 : INFO : precomputing L2-norms of word weight vectors


('亚洲', 0.6185275316238403)
('全世界', 0.5958456993103027)
('我国', 0.5894688963890076)
('全球', 0.5791220664978027)
('欧洲', 0.5742227435112)
('海外', 0.5666359663009644)
('GTI', 0.560455858707428)
('日本', 0.5581508874893188)
('中外', 0.5518802404403687)
('东南亚', 0.5481557846069336)


D:\opt\anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [13]:
word_similar = model.wv.most_similar("体育",topn=10)
for wv in word_similar:
    print(wv)

('社团', 0.7288618087768555)
('篮球', 0.7183200120925903)
('OCA', 0.7174619436264038)
('体育产业', 0.7131071090698242)
('智美', 0.7096304893493652)
('体育赛事', 0.6921861171722412)
('幼儿', 0.6896519660949707)
('ELITE12', 0.6831167340278625)
('篮球赛', 0.6772657632827759)
('电子竞技', 0.673592209815979)


D:\opt\anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### 计算词向量

In [14]:
model.wv.get_vector("中国")

array([-1.1522624 , -0.13746184,  1.1977522 , -1.7664257 , -1.2219906 ,
       -0.13999386,  3.8569906 ,  1.9267634 , -0.2056719 ,  1.4515932 ,
       -0.26038316,  0.7853203 ,  1.5362432 ,  1.612691  , -0.49071452,
        0.09172805,  1.0166246 , -0.7725363 , -2.1228657 ,  1.8180988 ,
       -0.7423364 ,  1.2573221 , -2.612258  , -0.1093672 , -0.05340657,
        0.4440791 , -2.867494  ,  2.4035149 ,  1.477893  ,  1.1256294 ,
        0.1267563 , -3.2468925 ,  0.02933542,  0.21708433, -0.6532297 ,
        1.1725346 , -0.45818368,  1.1296314 , -2.5563946 ,  1.0510018 ,
       -0.4650817 , -0.70601475,  1.609785  , -2.0332994 , -0.7391503 ,
        0.9180342 , -0.10604601, -0.44434017,  1.4818689 ,  1.1785319 ,
       -0.8409995 ,  1.7344096 ,  0.25315493, -0.7753752 , -0.4193366 ,
        1.5070014 ,  0.7722693 , -1.2543797 ,  0.13457158, -0.33472705,
        0.3681033 ,  1.5747145 , -1.0378097 , -1.2138793 , -1.4531838 ,
        2.1527858 , -0.57833695,  0.03880399,  0.59410256,  0.83

关于句子相似度，我们可以通过jieba分词，然后计算平均的每个词的向量，然后通过cos计算相似度

## 计算句子相似

In [23]:
import numpy as np

In [24]:
# 对每个句子的所有词向量取均值，来生成一个句子的vector
def build_sentence_vector(text, size, imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [30]:
dim=100
text1="金牌 班底 保证 品质"
text2="同质化 谈话 节目 落后 生产力"
text_1_vector=build_sentence_vector(text1.split(),dim,model)
text_2_vector=build_sentence_vector(text2.split(),dim,model)

D:\opt\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [31]:
text_1_vector

array([[-0.46178815,  0.54263485,  0.342274  ,  0.33006503,  0.57206069,
        -0.16667145,  0.26062876, -0.03352004, -0.49800857, -0.27923173,
        -0.54460798,  0.45641993, -0.86080161,  0.27758218, -1.20197947,
        -0.85763153,  0.18923374,  0.56686543, -0.13618138, -0.55435745,
        -0.04517515, -0.00288211, -0.51541702, -0.30828042,  0.38201458,
        -0.35516161, -0.63815296, -1.00554959,  0.2028668 ,  0.30985681,
        -0.66128606, -1.29287154,  0.71256616, -0.21823826, -1.27554249,
         0.07252802, -0.77478203,  0.39762783,  0.04207839,  0.23404703,
         0.0095697 ,  0.44164924,  0.59939779, -0.6150204 ,  0.71697787,
        -0.15711732,  0.75601466, -0.36745521,  0.38619499,  0.35793606,
         1.27060813,  1.03245136, -0.66065533,  0.24219526, -0.05416447,
        -0.2166793 ,  0.29253173,  0.5787945 ,  0.35621924,  0.63288261,
         0.85758192, -0.03926025,  0.02800246, -0.26811657, -0.29975752,
         0.72933676,  0.8842163 , -0.05754145,  0.6

In [33]:
text_2_vector

array([[ 0.19758285,  0.14212382,  0.25690537, -0.29763222,  0.60107128,
         0.01719552, -0.11725751,  0.09392206, -0.44982149, -0.07735251,
        -0.59122791,  0.02265279, -0.22360392,  0.49646062, -0.52764289,
        -0.61612995,  0.56112498, -0.49715714, -0.26017753, -0.82724852,
        -0.22608847,  0.78778634,  0.32907611,  0.14067345, -0.21323277,
        -0.26892053,  0.22976173, -0.0966771 ,  0.05375936,  0.75196879,
        -0.74821596, -0.47816626,  0.66834318, -0.09158119, -0.84514935,
        -0.20762842,  0.20921763,  0.22566281, -1.14158846,  0.38871826,
        -0.30845115,  0.30048392,  0.88220437,  0.25662818,  0.52279129,
         0.00645755,  0.41170232,  0.50595607, -0.23230699, -0.28025027,
         0.65846098,  0.38681295, -0.68398157,  0.00134029,  0.17166461,
         0.73254093,  0.05251779,  0.04331314, -0.10223302,  0.08562548,
        -0.15791062,  0.0161509 ,  0.32867494,  0.13696693,  0.28452277,
        -0.02343155,  0.19672216, -0.29948106,  0.2

下面的过程就不描述了，我们通过cos公式就可以计算出来了

## 总结，word2vec 和 fasttext 区别

https://blog.csdn.net/cdyx369/article/details/80579700 <br>
https://www.jianshu.com/p/da0a5edeca3d<br>
https://www.sohu.com/a/198733424_642762<br>

### fastText 原理
语言来说语序是很重要的，‘我爱你’的词袋特征<br>

在CBOW窗口内加入ngram特征（比如2gram）就会得到额外的信息，第一个句子的2gram特征是‘我爱’和‘爱你’，第二个句子的特征是‘你爱’ 和‘爱我’，这样就把上下文完全相同的两个句子区分开了，这种做法就是fastText的做法<br>

fastText简单来说就是将句子中的每个词先通过一个lookup层映射成词向量，然后对词向量取平均作为真个句子的句子向量，然后直接用线性分类器进行分类，从而实现文本分类.<br>

fastText完全是线性的，没有非线性隐藏层，得到的结果和有非线性层的网络差不多.对于句子结构简单的文本来说，但是这种方法显然没有考虑词序信息.fastText就不如有隐藏层等非线性结构的网络效果好
‘The movie is not very good , but i still like it . ’  <br>
‘The movie is very good , but i still do not like it .’ <br>
‘I do not like it , but the movie is still very good .’ <br>
这几个句子的词序差不多，用到的词也差不多，但是表达的意思是完全相反的，如果直接把词向量取平均，显然得到的平均词向量也是相差不到，在经过线性分类器分类很容易把这两个不同的类别分到同一类里，所以fastText很难学出词序对句子语义的影响，对复杂任务还是需要用复杂网络学习任务的语义表达。 



对于长文本的文本分类任务来说，就算是用词袋模型，效果也不差<br>
在Word2vec的基础上，把N-grams也当做词来训练word2vec，最后每个词的向量将由这个Ngrams得出。这个改进能提升模型对morphology的效果, 即”字面上”相似的词语distance也会小一些.<br>

(1)fastText包含三部分：模型结构，层次softmax，ngram  <br>
(2)fastText和Word2vector的CBOW模型很相似，不同的是fastText预测句子标签，CBOW预测中间词 <br>
(3)fastText的Ngram特征  <br>
词向量训练中常用的特征是词袋模型，但是词袋模型不能引入词序信息，比如‘我爱你’的词袋特征是‘我’ ‘爱’ ‘你’ ，‘你爱我’的词袋特征是‘你’ ‘爱’ ‘我’，这两个句子的特征是完全相同的。如果加入Ngram，这两个句子的特征就不同了，‘我爱你’的特征由加入了‘我爱’和‘爱你’，你爱我的特征有加入了‘你爱’和‘爱我’，用Ngram得到的特征是完全不同的，这两句话就能区别开了,当然，‘你爱’和‘爱我’这两个词也要包含在词典里，所以会有词频较少的词要舍弃，否则计算量太大的问题。 <br>


隐藏表征<br>
在不同类别所有分类器中进行共享，使得文本信息在不同类别中能够共同使用。这类表征被称为词袋（bag of words）（此处忽视词序）。在 fastText中也使用向量表征单词 n-gram来将局部词序考虑在内，这对很多文本分类问题来说十分重要。

### 层次softmax（hierarchical Softmax） 
层次softmax可以提高训练速度

### fastText与CBOW的区别 
fastText的模型和CBOW的模型结构一样，虽然结构一样，但是仍有不同 <br>
一、目的不一样，fastText是用来做文本分类的，虽然中间也会产生词向量，但词向量是一个副产物，而CBOW就是专门用来训练词向量的工具。 <br>
fastText的输出层是预测句子的类别标签，而CBOW的输出层是预测中间词； <br>
fastText的输入层是一个句子的每个词以及句子的ngram特征<br>
fastText是一个文本分类算法，是一个有监督模型，有额外标注的标签 。<br>
CBOW是一个训练词向量的算法，是一个无监督模型，没有额外的标签，其标准是语料本身，无需额外标注。<br>

fastText做文本分类的关键点是极大地提高了训练速度（在要分类的文本类别很多的情况下，比如500类），原因是在输出层采用了层级softmax，层级softmax如何提高训练速度在上面CBOW的层级softmax中已经介绍了，在这里就是将叶节点有词频变成文本分类数据集中每种类别的样本数量，霍夫曼树的结构也可以处理类别不均衡的问题（每种类别的样本数目不同），频繁出现类别的树形结构的深度要比不频繁出现类别的树形结构的深度要小，这也使得进一步的计算效率更高（意思是数目多的样本深度小，那很多样本都只需乘一次就把概率计算出来了，自然就快）。

## skip-model 
skip-gram模型是已知中间词，预测他的上下文
为提高训练速度，输出层仍是采用层次softmax，构造霍夫曼树来做，除了构造霍夫曼树，word2vec还提出了一种提高训练速度的方法，叫做负采样。

### 负采样（Negative Sampling） 
目的是提高训练速度并改善所得词向量的质量，以CBOW中的负采样为例，目标词w是正样本，其他的词就是负样本了

## word2vec 
word2vec是Google与2013年开源推出的一个用于获取word vecter的工具包，利用神经网络为单词寻找一个连续向量表示。

word2vec有两种网络模型，分别为：
* Continous Bag of Words Model （CBOW）- 根据上下文，预测中间词语
* Skip-Gram-根据中间词，预测上下文


## FastText词向量与word2vec对比

FastText= word2vec中 cbow + h-softmax的灵活使用

灵活体现在两个方面： 
1. 模型的输出层：word2vec的输出层，对应的是每一个term，计算某term的概率最大；而fasttext的输出层对应的是 分类的label。不过不管输出层对应的是什么内容，起对应的vector都不会被保留和使用； 
2. 模型的输入层：word2vec的输出层，是 context window 内的term；而fasttext 对应的整个sentence的内容，包括term，也包括 n-gram的内容；

两者本质的不同，体现在 h-softmax的使用。 
Wordvec的目的是得到词向量，该词向量 最终是在输入层得到，输出层对应的 h-softmax 也会生成一系列的向量，但最终都被抛弃，不会使用。 

fasttext则充分利用了h-softmax的分类功能，遍历分类树的所有叶节点，找到概率最大的label（一个或者N个）
